# 📋 Notebook 04 - Sistema de Reportes Automáticos

**DomusAI - Sprint 5**



## 1. 🔧 Configuración Inicial

Importar librerías necesarias y configurar entorno.

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Sistema de reportes DomusAI
import sys
sys.path.append('..')
from src.reporting import ReportGenerator, generate_quick_report

# Configurar visualizaciones
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configurar pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.3f}'.format)

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha actual: {datetime.now().strftime('%d/%m/%Y %H:%M')}")

## 2. 📂 Cargar Dataset Limpio

Cargamos el dataset ya procesado con `data_cleaning.py`.

In [ ]:
# Cargar datos limpios
data_path = Path('../data/Dataset_clean_test.csv')

if not data_path.exists():
    print("❌ Error: Dataset no encontrado")
    print("   Ejecuta primero: python src/data_cleaning.py")
else:
    df = pd.read_csv(data_path, parse_dates=['Datetime'], index_col='Datetime')
    
    print("✅ Dataset cargado correctamente")
    print(f"   📊 Registros: {len(df):,}")
    print(f"   📅 Período: {df.index.min().strftime('%d/%m/%Y')} - {df.index.max().strftime('%d/%m/%Y')}")
    print(f"   📁 Tamaño: {data_path.stat().st_size / (1024*1024):.2f} MB")
    print(f"\n   Columnas: {list(df.columns)}")

In [ ]:
# Vista previa de los datos
print("📊 Primeros registros del dataset:\n")
display(df.head(10))

print("\n📈 Estadísticas descriptivas:\n")
display(df.describe())

## 3. 🔍 Exploración del Sistema de Reportes

Vamos a explorar la clase `ReportGenerator` y sus capacidades.

In [ ]:
# Inicializar generador de reportes
generator = ReportGenerator(
    template_dir='../reports/templates',
    assets_dir='../reports/assets',
    output_dir='../reports/generated'
)

print("✅ ReportGenerator inicializado")
print(f"   📁 Templates: {generator.template_dir}")
print(f"   🎨 Assets: {generator.assets_dir}")
print(f"   💾 Output: {generator.output_dir}")

# Listar métodos disponibles
print("\n📋 Métodos principales:")
methods = [m for m in dir(generator) if not m.startswith('_') and callable(getattr(generator, m))]
for i, method in enumerate(methods, 1):
    print(f"   {i}. {method}()")

## 4. 📊 Análisis de Datos por Mes

Identificar meses disponibles en el dataset para generar reportes.

In [ ]:
# Análisis de datos por mes
# Usar pd.Grouper para agrupar por año y mes de forma robusta
monthly_summary = df.groupby([pd.Grouper(freq='M')]).agg({
    'Global_active_power': ['count', 'mean', 'sum', 'std']
})

monthly_summary.columns = ['Registros', 'Potencia_Media_kW', 'Potencia_Total_kW', 'Desv_Std']
monthly_summary['Consumo_kWh'] = monthly_summary['Potencia_Total_kW'] / 60  # Convertir a kWh

# Calcular días con datos por mes
monthly_summary['Dias'] = df.resample('M')['Global_active_power'].count() / 1440

# Extraer año y mes para el índice con cast explícito
idx = pd.DatetimeIndex(monthly_summary.index)
monthly_summary['Año'] = idx.year
monthly_summary['Mes'] = idx.month
monthly_summary = monthly_summary.set_index(['Año', 'Mes'])

print("📅 Datos disponibles por mes:\n")
display(monthly_summary)

print(f"\n✅ Total de meses con datos: {len(monthly_summary)}")

In [ ]:
# Visualizar consumo mensual
fig, ax = plt.subplots(figsize=(14, 6))

# Crear etiquetas de meses desde el índice multi-nivel (Año, Mes)
months_labels = [f"{year}/{month:02d}" for year, month in monthly_summary.index]
consumo = monthly_summary['Consumo_kWh'].to_numpy()  # Convertir a numpy array

# Crear barras
bars = ax.bar(range(len(consumo)), consumo, color='#667eea', alpha=0.8, edgecolor='#764ba2', linewidth=2)

# Colorear barra con mayor consumo
max_idx = np.argmax(consumo)
bars[max_idx].set_color('#e74c3c')
bars[max_idx].set_label('Mayor consumo')

# Línea de tendencia
z = np.polyfit(range(len(consumo)), consumo, 1)
p = np.poly1d(z)
ax.plot(range(len(consumo)), p(range(len(consumo))), "--", color='#764ba2', linewidth=2, label='Tendencia')

# Configuración del gráfico
ax.set_xlabel('Mes', fontsize=12, fontweight='600')
ax.set_ylabel('Consumo (kWh)', fontsize=12, fontweight='600')
ax.set_title('Consumo Energético Mensual - Dataset Completo', fontsize=16, fontweight='bold', pad=20)
ax.set_xticks(range(len(months_labels)))
ax.set_xticklabels(months_labels, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print(f"\n📊 Mes con mayor consumo: {months_labels[max_idx]} ({consumo[max_idx]:.2f} kWh)")

## 5. 🎯 Generación de Reporte Básico

Generar un reporte simple para el mes con más datos (Junio 2007).

In [ ]:
# Generar reporte para Junio 2007
print("📊 Generando reporte para Junio 2007...\n")

result = generator.generate_monthly_report(
    data=df,
    predictions=None,
    anomalies=None,
    month=6,
    year=2007
)

print("\n" + "="*80)
if result['status'] == 'success':
    print("✅ REPORTE GENERADO EXITOSAMENTE")
    print("="*80)
    print(f"\n📄 Archivo HTML: {result['html_path']}")
    print(f"⏱️  Tiempo de generación: {result['generation_time']:.2f}s")
    
    if result['charts']:
        print(f"\n📊 Gráficos generados:")
        for name, path in result['charts'].items():
            print(f"   • {name}: {path}")
else:
    print(f"❌ ERROR: {result.get('error')}")
    print("="*80)

## 6. 📈 Análisis de KPIs Calculados

Analizar los KPIs que calcula automáticamente el sistema.

In [ ]:
# Extraer resumen ejecutivo del reporte
summary = result['summary']

print("📊 RESUMEN EJECUTIVO - Junio 2007")
print("="*80)
print(f"\n🔢 KPIs Principales:")
print(f"   • Consumo Total:      {summary['total_consumption']:.2f} kWh")
print(f"   • Promedio Diario:    {summary['daily_avg']:.3f} kW")
print(f"   • Máximo Diario:      {summary['daily_max']:.3f} kW")
print(f"   • Mínimo Diario:      {summary['daily_min']:.3f} kW")
print(f"   • Cambio vs Anterior: {summary['change_pct']:+.1f}%")
print(f"   • Score Eficiencia:   {summary['efficiency_score']}/100")

print(f"\n📅 Período Analizado:")
print(f"   • Días completos:     {summary['period_days']}")
print(f"   • Total registros:    {summary['total_records']:,}")

print(f"\n⚠️  Anomalías:")
print(f"   • Total detectadas:   {summary['total_anomalies']}")
print(f"   • Críticas:           {summary['critical_anomalies']}")

In [ ]:
# Visualizar KPIs en tarjetas
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Dashboard de KPIs - Junio 2007', fontsize=18, fontweight='bold', y=0.98)

# KPI 1: Consumo Total
ax1 = axes[0, 0]
ax1.text(0.5, 0.6, f"{summary['total_consumption']:.1f}", 
         ha='center', va='center', fontsize=48, fontweight='bold', color='#667eea')
ax1.text(0.5, 0.3, 'kWh', ha='center', va='center', fontsize=20, color='#95a5a6')
ax1.text(0.5, 0.1, 'Consumo Total', ha='center', va='center', fontsize=14, fontweight='600')
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')
ax1.set_facecolor('#f8f9fa')

# KPI 2: Promedio Diario
ax2 = axes[0, 1]
ax2.text(0.5, 0.6, f"{summary['daily_avg']:.3f}", 
         ha='center', va='center', fontsize=48, fontweight='bold', color='#2ecc71')
ax2.text(0.5, 0.3, 'kW', ha='center', va='center', fontsize=20, color='#95a5a6')
ax2.text(0.5, 0.1, 'Promedio Diario', ha='center', va='center', fontsize=14, fontweight='600')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')
ax2.set_facecolor('#f8f9fa')

# KPI 3: Cambio %
ax3 = axes[1, 0]
color = '#e74c3c' if summary['change_pct'] > 0 else '#2ecc71'
symbol = '↑' if summary['change_pct'] > 0 else '↓'
ax3.text(0.5, 0.6, f"{abs(summary['change_pct']):.1f}% {symbol}", 
         ha='center', va='center', fontsize=44, fontweight='bold', color=color)
ax3.text(0.5, 0.3, 'vs mes anterior', ha='center', va='center', fontsize=16, color='#95a5a6')
ax3.text(0.5, 0.1, 'Cambio Mensual', ha='center', va='center', fontsize=14, fontweight='600')
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')
ax3.set_facecolor('#f8f9fa')

# KPI 4: Score Eficiencia
ax4 = axes[1, 1]
score = summary['efficiency_score']
color = '#2ecc71' if score >= 70 else '#f39c12' if score >= 50 else '#e74c3c'
ax4.text(0.5, 0.6, f"{score}", 
         ha='center', va='center', fontsize=48, fontweight='bold', color=color)
ax4.text(0.5, 0.3, '/100', ha='center', va='center', fontsize=20, color='#95a5a6')
ax4.text(0.5, 0.1, 'Score Eficiencia', ha='center', va='center', fontsize=14, fontweight='600')
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')
ax4.set_facecolor('#f8f9fa')

plt.tight_layout()
plt.show()

## 7. 📊 Análisis Detallado del Mes

Explorar patrones de consumo en el mes reportado.

In [ ]:
# Filtrar datos de Junio 2007
# Usar cast explícito para evitar warnings de tipo
idx = pd.DatetimeIndex(df.index)
mask = (idx.month == 6) & (idx.year == 2007)
june_data = df[mask]

print(f"📅 Datos de Junio 2007:")
print(f"   • Registros: {len(june_data):,}")
print(f"   • Período: {june_data.index.min()} a {june_data.index.max()}")
print(f"   • Días completos: {len(june_data) / 1440:.1f}")

In [ ]:
# Gráfico 1: Consumo diario con estadísticas
fig, ax = plt.subplots(figsize=(15, 6))

daily = june_data['Global_active_power'].resample('D').mean()

# Plot principal - Convertir a numpy array para compatibilidad
ax.plot(daily.index, daily.to_numpy(), linewidth=2.5, color='#667eea', 
        marker='o', markersize=6, label='Consumo Diario')

# Media móvil 7 días
ma7 = daily.rolling(window=7).mean()
ax.plot(ma7.index, ma7.to_numpy(), linewidth=2, linestyle='--', 
        color='#764ba2', alpha=0.7, label='Media Móvil 7 días')

# Líneas de referencia
ax.axhline(y=daily.mean(), color='#95a5a6', linestyle=':', linewidth=1.5, 
           label=f'Promedio: {daily.mean():.3f} kW')
ax.axhline(y=daily.quantile(0.90), color='#e74c3c', linestyle=':', linewidth=1.5, alpha=0.5,
           label=f'P90: {daily.quantile(0.90):.3f} kW')

# Marcar días con consumo alto
high_days = daily[daily > daily.quantile(0.90)]
ax.scatter(high_days.index, high_days.to_numpy(), color='#e74c3c', s=100, 
           marker='o', zorder=5, label='Consumo Alto (>P90)')

ax.set_title('Análisis de Consumo Diario - Junio 2007', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Fecha', fontsize=12, fontweight='600')
ax.set_ylabel('Potencia Media (kW)', fontsize=12, fontweight='600')
ax.legend(loc='upper right', framealpha=0.9)
ax.grid(True, alpha=0.3, linestyle='--')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico 2: Patrón horario (heatmap)
fig, ax = plt.subplots(figsize=(15, 8))

# Crear matriz de consumo por hora y día
hourly_data = june_data['Global_active_power'].copy()
hourly_data = hourly_data.to_frame()

# Cast explícito para acceder a atributos de fecha
idx = pd.DatetimeIndex(hourly_data.index)
hourly_data['Hour'] = idx.hour
hourly_data['Day'] = idx.day

pivot = hourly_data.pivot_table(
    values='Global_active_power',
    index='Hour',
    columns='Day',
    aggfunc='mean'
)

# Crear heatmap
sns.heatmap(pivot, cmap='YlOrRd', annot=False, fmt='.2f', 
            cbar_kws={'label': 'Potencia (kW)'}, ax=ax)

ax.set_title('Heatmap de Consumo por Hora y Día - Junio 2007', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Día del Mes', fontsize=12, fontweight='600')
ax.set_ylabel('Hora del Día', fontsize=12, fontweight='600')

plt.tight_layout()
plt.show()

print("\n🔥 Horas con mayor consumo promedio:")
peak_hours = pivot.mean(axis=1).nlargest(5)
for hour, power in peak_hours.items():
    print(f"   • {hour:02d}:00 - {power:.3f} kW")

In [ ]:
# Gráfico 3: Consumo por día de la semana
fig, ax = plt.subplots(figsize=(12, 6))

june_data_copy = june_data.copy()

# Cast explícito para acceder a atributos de fecha
idx = pd.DatetimeIndex(june_data_copy.index)
june_data_copy['DayOfWeek'] = idx.dayofweek
june_data_copy['DayName'] = idx.day_name()

# Calcular promedio por día de la semana
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_names_es = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

daily_avg = june_data_copy.groupby('DayName')['Global_active_power'].mean()
daily_avg = daily_avg.reindex(day_order)

# Plot con colores diferenciados - Convertir a numpy array
colors = ['#667eea'] * 5 + ['#e74c3c', '#f39c12']  # Weekdays azul, Sábado rojo, Domingo naranja
bars = ax.bar(day_names_es, daily_avg.to_numpy(), color=colors, alpha=0.8, edgecolor='#34495e', linewidth=2)

# Línea de promedio
ax.axhline(y=daily_avg.mean(), color='#95a5a6', linestyle='--', linewidth=2, 
           label=f'Promedio: {daily_avg.mean():.3f} kW')

ax.set_title('Consumo Promedio por Día de la Semana - Junio 2007', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Día de la Semana', fontsize=12, fontweight='600')
ax.set_ylabel('Potencia Media (kW)', fontsize=12, fontweight='600')
ax.legend()
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

print(f"\n📊 Día con mayor consumo: {day_names_es[daily_avg.argmax()]} ({daily_avg.max():.3f} kW)")
print(f"📊 Día con menor consumo: {day_names_es[daily_avg.argmin()]} ({daily_avg.min():.3f} kW)")

## 8. 💡 Sistema de Recomendaciones

Analizar las recomendaciones generadas automáticamente.

In [ ]:
# Generar recomendaciones para el mes
recommendations = generator.generate_recommendations(
    data=june_data,
    summary=summary,
    anomalies=None
)

print("💡 RECOMENDACIONES AUTOMÁTICAS")
print("="*80)

for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec['title']}")
    print(f"   {rec['description']}")
    if rec.get('savings'):
        print(f"   💰 Ahorro estimado: {rec['savings']}")
    print()

## 9. 📊 Generación de Múltiples Reportes

Generar reportes para todos los meses disponibles.

In [ ]:
# Generar reportes para todos los meses con datos suficientes
print("📊 Generando reportes para todos los meses...\n")

results = []

# Iterar sobre el índice multi-nivel correctamente
for idx, row in monthly_summary.iterrows():
    # Desempaquetar el índice multi-nivel
    year = int(idx[0]) if isinstance(idx, tuple) else int(idx)
    month = int(idx[1]) if isinstance(idx, tuple) and len(idx) > 1 else int(row.name[1] if hasattr(row, 'name') else idx)
    
    # Solo generar si hay al menos 20 días de datos
    if row['Dias'] >= 20:
        print(f"   Procesando {month:02d}/{year}...", end=" ")
        
        try:
            result = generator.generate_monthly_report(
                data=df,
                predictions=None,
                anomalies=None,
                month=month,
                year=year
            )
            
            if result['status'] == 'success':
                print(f"✅ ({result['generation_time']:.1f}s)")
                results.append({
                    'year': year,
                    'month': month,
                    'html_path': result['html_path'],
                    'time': result['generation_time']
                })
            else:
                print(f"❌ Error")
        except Exception as e:
            print(f"❌ Excepción: {e}")

print(f"\n✅ {len(results)} reportes generados exitosamente")
print(f"⏱️  Tiempo total: {sum(r['time'] for r in results):.2f}s")
print(f"⚡ Promedio por reporte: {sum(r['time'] for r in results)/len(results):.2f}s")

In [ ]:
# Mostrar lista de reportes generados
print("\n📄 Reportes Generados:")
print("="*80)

for i, r in enumerate(results, 1):
    print(f"{i}. {r['month']:02d}/{r['year']} - {Path(r['html_path']).name}")
    print(f"   ⏱️  {r['time']:.2f}s")

## 10. 📈 Comparativa de Reportes

Analizar evolución de KPIs entre meses.

In [ ]:
# Generar comparativa de KPIs entre meses
kpis_comparison = []

# Usar cast explícito para evitar warnings de tipo
idx_df = pd.DatetimeIndex(df.index)

# Iterar sobre el índice multi-nivel correctamente
for idx, row in monthly_summary.iterrows():
    # Desempaquetar el índice multi-nivel
    year = int(idx[0]) if isinstance(idx, tuple) else int(idx)
    month = int(idx[1]) if isinstance(idx, tuple) and len(idx) > 1 else int(row.name[1] if hasattr(row, 'name') else idx)
    
    if row['Dias'] >= 20:
        # Calcular KPIs para cada mes
        mask = (idx_df.month == month) & (idx_df.year == year)
        month_data = df[mask]
        
        summary_temp = generator.create_executive_summary(df, month, year)
        
        kpis_comparison.append({
            'Año': year,
            'Mes': month,
            'Consumo_kWh': summary_temp['total_consumption'],
            'Promedio_kW': summary_temp['daily_avg'],
            'Eficiencia': summary_temp['efficiency_score'],
            'Cambio_%': summary_temp['change_pct']
        })

kpis_df = pd.DataFrame(kpis_comparison)
kpis_df['Período'] = kpis_df.apply(lambda x: f"{x['Mes']:02d}/{x['Año']}", axis=1)

print("📊 Comparativa de KPIs entre Meses:\n")
display(kpis_df)

In [ ]:
# Visualización de evolución de KPIs
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Evolución de KPIs - Todos los Meses', fontsize=18, fontweight='bold', y=0.98)

# Gráfico 1: Consumo Total
ax1 = axes[0, 0]
ax1.plot(kpis_df['Período'], kpis_df['Consumo_kWh'], marker='o', linewidth=2.5, color='#667eea')
ax1.set_title('Consumo Total Mensual', fontsize=14, fontweight='bold')
ax1.set_xlabel('Mes')
ax1.set_ylabel('kWh')
ax1.grid(alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Gráfico 2: Promedio Diario
ax2 = axes[0, 1]
ax2.plot(kpis_df['Período'], kpis_df['Promedio_kW'], marker='s', linewidth=2.5, color='#2ecc71')
ax2.set_title('Promedio Diario', fontsize=14, fontweight='bold')
ax2.set_xlabel('Mes')
ax2.set_ylabel('kW')
ax2.grid(alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# Gráfico 3: Cambio Porcentual
ax3 = axes[1, 0]
colors = ['#e74c3c' if x > 0 else '#2ecc71' for x in kpis_df['Cambio_%']]
ax3.bar(kpis_df['Período'], kpis_df['Cambio_%'], color=colors, alpha=0.8, edgecolor='#34495e', linewidth=1.5)
ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
ax3.set_title('Cambio % vs Mes Anterior', fontsize=14, fontweight='bold')
ax3.set_xlabel('Mes')
ax3.set_ylabel('%')
ax3.grid(alpha=0.3, axis='y')
ax3.tick_params(axis='x', rotation=45)

# Gráfico 4: Score Eficiencia
ax4 = axes[1, 1]
colors = ['#2ecc71' if x >= 70 else '#f39c12' if x >= 50 else '#e74c3c' for x in kpis_df['Eficiencia']]
ax4.bar(kpis_df['Período'], kpis_df['Eficiencia'], color=colors, alpha=0.8, edgecolor='#34495e', linewidth=1.5)
ax4.axhline(y=70, color='#2ecc71', linestyle='--', linewidth=1.5, label='Objetivo (70)')
ax4.set_title('Score de Eficiencia', fontsize=14, fontweight='bold')
ax4.set_xlabel('Mes')
ax4.set_ylabel('Score (0-100)')
ax4.set_ylim(0, 100)
ax4.legend()
ax4.grid(alpha=0.3, axis='y')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()